In [1]:
import os
# get current working directory
os.getcwd()
import sys
sys.path.append("..")
!pip install fire
!pip install torch
!pip install langchain
!pip install accelerate
!pip install datasets
!pip install transformers
!pip install openai


In [2]:
from tasks.legalbench import LBTask
from datasets import load_dataset
from eval_llm import EvalLLM, LLMConfig
from datetime import datetime
import pandas as pd

In [3]:
os.environ["OPENAI_API_KEY"] = "sk-xxxxx"

In [4]:
task_dir = "legalbench"
# gets all legalbench tasks
tasks = [
name
for name in os.listdir(task_dir)
if os.path.isdir(os.path.join(task_dir, name))
]
#tasks = tasks[11:17]
tasks = tasks[1:2]
print(tasks)

['cuad_governing_law']


In [5]:
# creates a file to store the results in
header = True
results_filename = str(datetime.now()) + "_RESULTS.csv"
open(results_filename, "w+").close()

In [6]:
config = LLMConfig(
        model_name="gpt-3.5-turbo",
        max_new_tokens=6,
        temperature=0.0,
        rate_limit=600,
)

In [7]:
for task_name in tasks:
    task = LBTask(task_name, train = False)
    
    data = task.prepare_data()
    
    llm = EvalLLM(config, task.prompt_template())
    completions = await llm.run(data)
    
    metrics = task.score(data, completions)
    print(metrics)
    
    # append task result to result_file
    results_df = pd.DataFrame({"test": [task_name], "metrics": [metrics]})
    results_df.to_csv(results_filename, mode="a", index=False, header=header)
    header = False

Generating completions with rate limit of 600 calls per minute


100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.28s/it]

Finished generating completions in 2.32 seconds
{'accuracy': 1.0}
